In [18]:
!pip install transformers
!pip install accelerate -U

In [19]:
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [20]:
from google.colab import files
uploaded = files.upload()

Saving codex_test.txt to codex_test (1).txt
Saving codex_train.txt to codex_train (1).txt


In [21]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'codex_train.txt'
test_path = 'codex_test.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [23]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [26]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps = 400,
    save_steps=800,
    warmup_steps=500
    )

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [28]:
trainer.train()

Step,Training Loss
500,1.582100


TrainOutput(global_step=830, training_loss=1.2066418383494917, metrics={'train_runtime': 159.4637, 'train_samples_per_second': 20.757, 'train_steps_per_second': 5.205, 'total_flos': 216219156480000.0, 'train_loss': 1.2066418383494917, 'epoch': 10.0})

In [29]:
trainer.save_model()

In [30]:
tokenizer.save_pretrained('gpt_chf')
model.save_pretrained('model_gpt_chf')

In [31]:
tokenizer = AutoTokenizer.from_pretrained("gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

In [32]:
prefix = "муж не имеет права"

In [33]:
tokens = tokenizer(prefix, return_tensors='pt')

In [37]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    do_sample=False,
    max_length=size+50,
    repetition_penalty=5.,
    temperature=0.3,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


муж не имеет права собственности на жилое помещение или право пользования жилым помещением, а также сохраняет
 имущественные права, основанные на факте родства с родителями и другими родственниками.
     (Пункт в редакции, введенной в действие с 10 января
